# Solution: Implementing a Sudoku solver and testing for uniqueness

## Modeling

<font color='blue'><b>First task:</b></font> Writing constraints for the variables $x_{ijk}$.

There are various different constraints that we have to include:

- **Respecting the given hints:** The given numbers in the Sudoku should remain as given, hence for any number $k$ given in cell $(i,j)$ of the sudoku, i.e., in row $i$ and column $j$, we put $x_{ijk}=1$ as a constraint.

- **Precisely one number per cell:** For every cell $(i,j)$ of the Sudoku, we should have precisely one variable $x_{ijk}$ set to $1$, while all others should be zero. For a binary solution, this is equivalent to having
   
   $$ \sum_{k=1}^9 x_{ijk} = 1\qquad\forall (i,j)\in[9]\times[9]\enspace. $$
   
- **No row contains two equal numbers:** For every number $k$ and every row $i$, there should be precisely one cell $(i,j)$ that contains the number $k$. For the binary variables $x_{ijk}$, this can be encoded in the constraint
   
   $$ \sum_{j=1}^9 x_{i,j,k} = 1 \qquad\forall (i,k)\in[9]\times[9]\enspace. $$
   
- **No column contains two equal numbers:** For every number $k$ and every column $j$, there should be precisely one cell $(i,j)$ that contains the number $k$. For the binary variables $x_{ijk}$, this can be encoded in the constraint
   
   $$ \sum_{i=1}^9 x_{i,j,k} = 1 \qquad\forall (j,k)\in[9]\times[9]\enspace. $$
   
- **No one of the nine $3\times 3$ squares that the grid can be partitioned in contains two equal numbers:** As above, this can be encoded as
   
   $$ \sum_{(i,j)\in C_\ell} x_{ijk} = 1 \qquad\forall (k,\ell)\in[9]\times[9]\enspace, $$
   
   where $C_1,\ldots,C_9$ denote the nine $3\times 3$ squares that the Sudoku grid is partitioned into.
   
Any solution of the Sudoku satisfies all these constraints, and we also see that any binary $x_{ijk}$ satisfying all constraints corresponds to a solution of the Sudoku.

---

## Implementing

<font color='blue'><b>Second task:</b></font> Implementation: One implementation is given below. Note that there is an optional argument `objective` taken by the Sudoku solver function below. For this task (i.e., the second task), anything related to the objective could be deleted from the implementation - we are only interested in solving the feasibility problem. Note that the default argument given for the `objective` parameter is such that the objective is set to zero, hence a call to the `sudokuSolver` function with no `objective` specified is indeed the same as just solving the feasibility problem.

In [ ]:
# Implementation of a Sudoku solver

def sudokuSolver(inputSudoku, objective = 81*[0]):
    
    ## helper function for mapping cells to serial index
    def ind(i,j):
        return 9*(i-1) + j-1
    
    
    ## setting up IP
    import pulp
    sudokuIP = pulp.LpProblem("Sudoku IP", pulp.LpMinimize)
    
    # variables
    var = [[[pulp.LpVariable(f"x_{i+1}{j+1}{k+1}", cat='Binary') 
             for k in range(9)] for j in range(9)] for i in range(9)]
    def x(i,j,k):
        return var[i-1][j-1][k-1]
    
    ## adding constraints
    for i in range(1,10):
        for j in range(1,10):
            # respect given hints
            if inputSudoku[ind(i,j)] != 0:
                sudokuIP += x(i,j,inputSudoku[ind(i,j)]) == 1.0
            # one number per cell
            sudokuIP += pulp.lpSum([x(i,j,k) for k in range(1,10)]) == 1.0
            
    # row and column constraints
    for i in range(1,10):
        for k in range(1,10):
            # row constraint
            sudokuIP += pulp.lpSum([x(i,j,k) for j in range(1,10)]) == 1.0
            # column constraint (note the interchanged role of i and j)
            sudokuIP += pulp.lpSum([x(j,i,k) for j in range(1,10)]) == 1.0
    
    # square constraints
    for ii in range(1,4):
        for jj in range(1,4):
            for k in range(1,10):
                sudokuIP += pulp.lpSum([x(i,j,k) for i in range(3*(ii-1)+1, 3*ii+1) 
                                        for j in range(3*(jj-1)+1, 3*jj+1)]) == 1.0
    
    # set objective
    sudokuIP += pulp.lpSum([x(i,j,objective[ind(i,j)]) for i in range(1,10)
                            for j in range(1,10) if objective[ind(i,j)] != 0])
            
    
    ## solve IP, return None if infeasible (corresponds to the Sudoku not having a solution)
    status = sudokuIP.solve()
    if status != 1:
        return None
    
    ## read solution
    outputSudoku = inputSudoku.copy()
    for i in range(1,10):
        for j in range(1,10):
            for k in range(1,10):
                if x(i,j,k).value() == 1.0:
                    outputSudoku[ind(i,j)] = k
    
    return outputSudoku

Using the above function, we can now solve `sudoku1`, for example.

In [ ]:
# Example Sudoku input and Sudoku printing function
sudoku1 = [4, 0, 7, 0, 0, 0, 0, 0, 0, 
           0, 3, 5, 0, 9, 7, 4, 0, 0, 
           0, 9, 0, 0, 0, 0, 0, 0, 6, 
           0, 0, 0, 3, 0, 2, 0, 0, 0, 
           6, 0, 0, 0, 8, 0, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 5, 0, 0, 
           0, 0, 0, 4, 0, 0, 0, 1, 8, 
           0, 0, 3, 0, 2, 8, 0, 0, 4, 
           5, 0, 4, 0, 0, 0, 0, 9, 7]

def printSudoku(sudoku):
    # compact Sudoku printing function
    # taken from https://codegolf.stackexchange.com/questions/126930/
    #    draw-a-sudoku-board-using-line-drawing-characters
    q = lambda x,y:x+y+x+y+x
    r = lambda a,b,c,d,e:a+q(q(b*3,c),d)+e+"\n"
    print(((r(*"╔═╤╦╗") + q(q("║ %d │ %d │ %d "*3 + "║\n",r(*"╟─┼╫╢")), r(*"╠═╪╬╣")) +
            r(*"╚═╧╩╝")) % tuple(sudoku)).replace(*"0 "))

In [ ]:
printSudoku(sudoku1)
sol1 = sudokuSolver(sudoku1)
printSudoku(sol1)

---

## Checking uniqueness

<font color='blue'><b>Third task:</b></font> Checking for the number of solutions. If there is no solution, then of course we detect this by our IP being infeasible. If there are solutions, we will find one using the IP, and to see if there exist more solutions, we can do a trick exploiting the objective function of our IP, which we currently did not use: Consider the support $S$ of a fixed solution of the IP. Any other solution will have a different support, i.e., the number of variables in $S$ that are set to one will be strictly less than $|S|$. Thus, putting the sum of all variables in $S$ as an objective and minimizing over all solutions will give a different solution if there exists one.

The Sudoku solver function above is already implemented in a way such that a second Sudoku (the parameter `objective`) can be provided (in the same format as the input Sudoku), and the objective will be set accordingly. Note that the `objective` sudoku can also be partial, i.e., it can have zero entries, which will be ignored. The latter allows, for example, to check whether an input sudoku has a solution with a number different from $5$ in the top left cell by using `objective = [5, 0, 0, ..., 0]`.

With this functionality, we can now easily implement the desired test.

In [ ]:
def numberOfSolutions(inputSudoku):
    
    ## Your code goes here.
    sol1 = sudokuSolver(inputSudoku)
    
    if sol1 == None:
        return (0, None)
    
    sol2 = sudokuSolver(inputSudoku, sol1)
    
    if sol1 == sol2:
        return (1, [sol1])
    
    return (2, [sol1, sol2])

---

## Testing

In [ ]:
sudoku2 = [2, 0, 0, 0, 0, 0, 0, 4, 0, 
           1, 0, 0, 0, 0, 0, 0, 0, 7,
           8, 0, 6, 3, 0, 0, 0, 0, 0,
           0, 5, 0, 0, 0, 7, 3, 0, 1, 
           0, 0, 3, 0, 1, 0, 0, 0, 0, 
           0, 0, 2, 0, 0, 3, 7, 5, 4, 
           0, 0, 7, 0, 0, 5, 0, 0, 0, 
           5, 0, 0, 0, 4, 0, 0, 0, 0, 
           0, 0, 0, 1, 7, 0, 0, 0, 8]

sudoku3 = [0, 0, 0, 6, 0, 7, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 0, 9, 8,
           3, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 2, 0, 6, 0, 0, 
           0, 0, 0, 0, 0, 0, 7, 0, 0, 
           0, 4, 0, 0, 8, 0, 0, 0, 0, 
           1, 0, 0, 0, 0, 0, 0, 2, 3, 
           0, 0, 8, 9, 0, 0, 0, 0, 0, 
           0, 0, 0, 4, 0, 0, 1, 0, 0]

sudoku4 = [0, 6, 0, 0, 0, 0, 0, 7, 4,
           1, 0, 0, 6, 0, 7, 0, 0, 3, 
           7, 0, 0, 0, 0, 0, 0, 0, 0, 
           0, 0, 0, 0, 1, 0, 0, 0, 2, 
           0, 0, 1, 5, 0, 0, 9, 0, 0, 
           9, 0, 0, 8, 0, 0, 0, 1, 0, 
           0, 0, 0, 0, 0, 0, 0, 3, 0, 
           3, 0, 0, 0, 0, 2, 8, 5, 0, 
           0, 0, 9, 0, 0, 4, 0, 0, 0]

In [ ]:
for sudoku in [sudoku2, sudoku3, sudoku4]:
    print("Input Sudoku:")
    printSudoku(sudoku)
    (n, solns) = numberOfSolutions(sudoku)
    print(f"Sudoku has {'no solution.' if n == 0 else 'one unique solution:' if n == 1 else 'at least two solutions:'}")
    if n > 0:
        for sol in solns:
            printSudoku(sol)
    print()
    print("*************************************")
    print()
